In [6]:
import pandas as pd
from sklearn.metrics import classification_report

In [7]:
bsq = [
    "Does the article discuss agroecological practices?",
    "Does the article focus primarily on climate change mitigation?",
    "Does the article cover climate change adaptation?",
    "Does the article comprehensively cover climate change and environmental aspects?",
    "Does the study specifically evaluate greenhouse gas emissions?",
    "Does the article assess agroecological practices' impact on climate change?",
    "Does the article target stakeholders?",
    "Does the article address limitations, challenges, and potential risks?",
    "Does the article address policy implications?",
    "Does the article have a specific geographic focus?",
    "Does the article discuss the role of agroecology in enhancing climate resilience in agricultural systems?",
    "Does the article assess the impact of agroecology on nitrogen dynamics?",
    "Does the article discuss the impact of methane (CH4) emissions?"
]
len(bsq)

13

In [1]:
chatgpt_responses_b = []
for i in range(len(bsq)):
    if i+1 in [7, 5, 2]:
        chatgpt_responses_b.append(pd.read_excel(f"chatgpt_responses_b{i+1}_v2.xlsx", index_col=0))
    else:
        o = pd.read_excel(f"chatgpt_responses_b{i+1}.xlsx", index_col=0)
        o_ = pd.read_excel(f"chatgpt_responses_b{i+1}_v2.xlsx", index_col=0)
        chatgpt_responses_b.append(pd.concat([o, o_]))
    print(i+1, bsq[i])
    print(classification_report(chatgpt_responses_b[-1]["Final decision"] == "INCLUDE", chatgpt_responses_b[-1]["chatgpt"].apply(lambda x: int("Yes" in x)), digits=4))
    print(chatgpt_responses_b[-1]["chatgpt"].apply(lambda x: int("Yes" in x)).value_counts().to_dict())
    print(chatgpt_responses_b[-1].shape)

NameError: name 'bsq' is not defined

In [10]:
to_pred = chatgpt_responses_b[0].copy()
to_pred["b1"] = chatgpt_responses_b[0]["chatgpt"].apply(lambda x: int("Yes" in x))
for i in range(0, len(bsq)):
    to_pred[f"b{i+1}"] = chatgpt_responses_b[i]["chatgpt"].apply(lambda x: int("Yes" in x))

In [11]:
X = to_pred[[f"b{i+1}" for i in range(len(bsq))]]
y = (to_pred["Final decision"] == "INCLUDE").apply(int)

In [12]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)

In [13]:
clf.fit(X, y)
clf.score(X, y)

0.8742138364779874

In [14]:
print(classification_report(y, clf.predict(X), digits=4))

              precision    recall  f1-score   support

           0     0.8696    0.9091    0.8889        88
           1     0.8806    0.8310    0.8551        71

    accuracy                         0.8742       159
   macro avg     0.8751    0.8700    0.8720       159
weighted avg     0.8745    0.8742    0.8738       159



In [17]:
row

Publication Type                                                    J
Authors              Reda, GK; Kebede, TG; Kahsay, ST; Gebrehiwot, BH
Article Title       Carbon sequestration and vegetation properties...
Abstract (AB)       Exclosure management is becoming a common appr...
Publication Year                                               2020.0
DOI                                         10.1016/j.jnc.2020.125856
Researcher1                                                       NaN
Researcher2                                                   EXCLUDE
Researcher3                                                   EXCLUDE
Final decision                                                EXCLUDE
Review studies                                                    NaN
Duplicates                                                        NaN
chatgpt                                                           Yes
b1                                                                  1
b2                  

In [16]:
row = to_pred[y != clf.predict(X)].sample(1).iloc[0]
print(f"""Title: {row["Article Title"]}\nAbstract: {row["Abstract (AB)"]}\nLabel: {row["Final decision"]}""")

Title: Carbon sequestration and vegetation properties across the age of community managed exclosures in Northern Ethiopia
Abstract: Exclosure management is becoming a common approach to rehabilitate and increase net biomass productivity of extremely degraded areas in Northern Ethiopia. However, the effect of age of exclosures on vegetation and soil property of communal lands is not yet well studied. In this study, the effect of land exclusion and age of exclosure on plant species richness, diversity, density and dominance, herbaceous standing biomass, woody biomass, woody biomass carbon, soil organic carbon (SOC) and other soil properties were assessed. To study the above listed variables, exclosures in three age classes (5-7, 12-15 and > 20 years) and adjacent free grazing areas were selected. We replicate each age class three times considering agroecological similarity. From each exclosure and free grazing area, six larger plots (10m x 20m) each having five small quadrats were taken.

In [18]:
columns = "Article Title  Abstract (AB)".split("  ")
recols = {"Article Title": "t", "Abstract (AB)": "ab"}
i = 0
o = chatgpt_responses_b[i][columns].rename(columns = recols)
o["q"] = bsq[i]
o["label"] = to_pred[f"b{i+1}"]
o = o.reset_index().drop(columns="index")
for i in range(1, len(bsq)):
    if i+1 not in [7]:
        oi = chatgpt_responses_b[i][columns].rename(columns = recols)
        oi["q"] = bsq[i]
        oi["label"] = to_pred[f"b{i+1}"]
        oi = oi.reset_index().drop(columns="index")
        o = pd.concat([o, oi], ignore_index=True)
o

,t,ab,q,label
0,Does carbon farming provide a cost-effective o...,"In this study, we apply a whole farm bioeconom...",Does the article discuss agroecological practi...,1
1,Influence of Organic and Mineral Fertilizers o...,The intensive use of mineral (M) fertilizers m...,Does the article discuss agroecological practi...,1
2,Climate Change Mitigation Options in the Fores...,The Intergovernmental Panel on Climate Change ...,Does the article discuss agroecological practi...,1
3,Rye cover crop incorporation and high watertab...,Drainage and cultivation of peat soils almost ...,Does the article discuss agroecological practi...,1
4,Emerging Issues and Potential Opportunities in...,The rice-wheat cropping system (RWCS) is the b...,Does the article discuss agroecological practi...,1
...,...,...,...,...
1903,Crop-livestock integration provides opportunit...,CONTEXT: The Greater Mekong Subregion has been...,Does the article discuss the impact of methane...,1
1904,Pesticide Use and Associated Greenhouse Gas Em...,The production of synthetic pesticides is ener...,Does the article discuss the impact of methane...,0
1905,"A Review: Soil Management, Sustainable Strateg...",Conservative and sustainable soil management i...,Does the article discuss the impact of methane...,0
1906,Agricultural waste recycling in horticultural ...,The vegetables supply chain of intensive farmi...,Does the article discuss the impact of methane...,0


In [19]:
o["label"].value_counts()

1    1291
0     617
Name: label, dtype: int64

In [20]:
o.to_excel("train_bsq_abstract_chatgpt_v2.xlsx")